**fsociety has done it, SG-Corp has been hacked and everyone is now penniless! Can our Singaporean Kaggle-rs still become a millionaire before retirement?**

Referencing the series "Mr.Robot", this notebook will focus on Singaporean Kagglers and see if they still can become a millionaire even though Elliot Alderson has wiped everyone's bank account. 

Several assumptions made while working on this project.

* The working age will be 21 (before University) (2 years of national service will not be factored into the working age for Singaporean males)
* The retirement age be 70
* This notebook assumes that "I have never written code" means < 1 years programming experience as well
* This notebook will take the "Years of coding experience" as working experience
* CPF and investments will not be factored into the calculation. #Let's just say Elliot has hacked all financial institutions as well.

Bread and butter imports

In [ ]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/kaggle-survey-2021/kaggle_survey_2021_responses.csv')
df.drop(0, inplace=True)
df.head()

Taking a look at the number of Singapore Kagglers submissions

In [ ]:
df['Q3'][df['Q3'] == 'Singapore'].count()

In [ ]:
#Get the Singporeans!
df_sg = df[df['Q3'] == 'Singapore']

Education background split among the Singaporeans

In [ ]:
plt.figure(figsize=(20,3))
plt.xticks(rotation=90)
sns.countplot(data=df_sg, x='Q4')

Drop 'students' and 'Currently not employed'(as we are assusming they do not have salary)
Drop 'Other' so we can limit to the jobs available in the survey

In [ ]:
df_sg = df_sg[(df_sg['Q5'] != 'Student') & (df_sg['Q5'] !='Other') & (df_sg['Q5'] !='Currently not employed')]
plt.figure(figsize=(20,3))
plt.xticks(rotation=90)
sns.countplot(data=df_sg, x='Q5')

Salary ranges

In [ ]:
plt.figure(figsize=(20,3))
plt.xticks(rotation=90)
sns.countplot(data=df_sg, x='Q25')

As there is quite a lot of rows with no or 0-999 salary, will replace the values with the mode of the relevant salaries.

In [ ]:

df_sg[(df_sg['Q25'] != '$0-999') | (pd.isna(df_sg['Q25']))]

#That ’ gave me headaches!! replacing with '

In [ ]:
df_sg['Q4'] = df_sg['Q4'].apply(lambda x: x.replace("’","'"))

The goal was to fill NA and 0 to 999 dollars values with the mode of the relevant fields using data from (Q3, Q4, Q5, Q6) which are (Country, Eduaction Level, Role, Years of Experience Programming) <br>
The method will minimally match the job title and exclude the ones with salary ranges of "0 to 999 dollars"

In [ ]:

def salary_fill_na(cols):
    if pd.notna(cols[4]):        
        if cols[4] != '$0-999':
            return cols[4]
    if pd.isna(df_sg[(df_sg['Q3'] == cols[0]) & (df_sg['Q4'] == cols[1]) & (df_sg['Q5'] == cols[2]) & (df_sg['Q6'] == cols[3]) & (df_sg['Q25'] != '$0-999')].groupby(df_sg['Q25']).count()["Q1"].max()):
        if pd.isna(df_sg[(df_sg['Q3'] == cols[0]) & (df_sg['Q4'] == cols[1]) & (df_sg['Q5'] == cols[2]) & (df_sg['Q25'] != '$0-999')].groupby(df_sg['Q25']).count()["Q1"].max()):
            if pd.isna(df_sg[(df_sg['Q3'] == cols[0]) & (df_sg['Q5'] == cols[2]) & (df_sg['Q25'] != '$0-999')].groupby(df_sg['Q25']).count()["Q1"].max()):
                return pd.isna(df_sg[(df_sg['Q5'] == cols[2]) & (df_sg['Q25'] != '$0-999')].groupby(df_sg['Q25']).count()["Q1"].idxmax())
            else:
                return df_sg[(df_sg['Q3'] == cols[0]) & (df_sg['Q5'] == cols[2]) & (df_sg['Q25'] != '$0-999')].groupby(df_sg['Q25']).count()["Q1"].idxmax()
        else:
            return df_sg[(df_sg['Q3'] == cols[0]) & (df_sg['Q4'] == cols[1]) & (df_sg['Q5'] == cols[2]) & (df_sg['Q25'] != '$0-999')].groupby(df_sg['Q25']).count()["Q1"].idxmax()
    else:
        return df_sg[(df_sg['Q3'] == cols[0]) & (df_sg['Q4'] == cols[1]) & (df_sg['Q5'] == cols[2]) & (df_sg['Q6'] == cols[3]) & (df_sg['Q25'] != '$0-999')].groupby(df_sg['Q25']).count()["Q1"].idxmax()

In [ ]:
df_sg['Q25'] = df_sg[['Q3','Q4','Q5','Q6','Q25']].apply(salary_fill_na, axis=1)

New salary range

In [ ]:

plt.figure(figsize=(20,3))
plt.xticks(rotation=90)
sns.countplot(data=df_sg, x='Q25')

Remove those with no relevant salaries to reference from

In [ ]:

df_sg = df_sg[df_sg['Q25'] != False]

convert to digits, combining < 1 years and I have never written code

In [ ]:
def convert_q6(x):
    if x=='20+ years':
        return 5
    elif x=='10-20 years':
        return 4
    elif x=='5-10 years':
        return 3
    elif x=='3-5 years':
        return 2
    elif x=='1-3 years':
        return 1
    else:
        return 0

In [ ]:
df_sg['Q6_code'] = df_sg['Q6'].apply(convert_q6)

Convert Q25 to values according to salary range and years of working.<br><br>

We will assume that the more years of working will mean that you will be on the higher quartile of the salary range.<br>
e.g. if salary range is 1000-5000 and,<br>
you have 1 year of working experience, your salary will be 1000<br>
if you have 20 years of working experience, your salary will be 5000<br>

In [ ]:
def convert_q25(cols):
    x = cols[1]
    if '-' in x:
        values = x.split('-')
        y = ''
        for i in values:
            for c in i:
                if c.isdigit():
                    y+=c
            y+='+'
        y = y[:-1].split('+')
        return int(y[0]) + ((int(y[1])-int(y[0]))/5)*cols[0]
    else:
        y = ''
        for c in x:
            if c.isdigit():
                y+=c
        return int(y)


In [ ]:
df_sg['Q25_code'] = df_sg[['Q6_code','Q25']].apply(convert_q25, axis=1)

In [ ]:
df_sg[['Q6','Q25']].head()

Lets take a look at the boxplot of the salary ranges now

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(rotation=90)
sns.boxplot(x='Q5', y='Q25_code', hue='Q6', data=df_sg)

“The 50-30-20 rule advocates splitting one's income into three categories – 50% on essentials, 30% on discretionary spending, and 20% on savings or investments.”<br>
Hence, we will use a savings rate of 0.2

In [ ]:
SAVINGS_RATE = 0.20

I am sure there's a way to do both functions together and apply to the right columns/rows. I got lazy to find out...

In [ ]:
def get_savings(x):
    savings = 0
    years = 0
    while savings < 1000000:
        savings+= SAVINGS_RATE*x
        years+=1
        #13-month bonus
        savings+=x/12
        if years%5==0:    
            #job promotion 15% increment
            x = x*1.15
        else:        
            #3.5% yearly salary increment
            x = x*1.035
        
    return savings

In [ ]:
def can_i_be_a_millionaire_just_by_working(x):
    savings = 0
    years = 0
    while savings < 1000000:
        savings+= SAVINGS_RATE*x
        years+=1
        #13-month bonus
        savings+=x/12
        if years%5==0: 
            #job promotion 15% increment
            x = x*1.15
        else:
             #3.5% yearly salary increment
            x = x*1.035
        
    return years

In [ ]:
df_sg['savings'] = df_sg['Q25_code'].apply(get_savings)
df_sg['millionaire_years'] = df_sg['Q25_code'].apply(can_i_be_a_millionaire_just_by_working)

The number of years to work before getting that million bucks

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(rotation=90)
sns.barplot(y='millionaire_years', x='Q5', hue='Q6', data=df_sg)

We will add the mean of the working years and the number of years it takes for the individual to reach their respective education levels to the working age of 21. <br>
And then add the number of years it will take them to save a million dollars <br>

In [ ]:
working_years = {
    '5-10 years':8, 
    '10-20 years':15, 
    '1-3 years':2,
    'I have never written code':0, 
    '3-5 years':4, 
    '20+ years':20, 
    '< 1 years':0
}

In [ ]:
education_years = {
    "Master's degree":6,
    "Bachelor's degree":4,
    'Doctoral degree':12,
    'No formal education past high school':0,
    "Some college/university study without earning a bachelor's degree":2,
    'Professional doctorate':12
}

In [ ]:
def i_am_the_millionaire(cols):
    working_age = 21
    working_age += working_years[cols[1]] 
    working_age += education_years[cols[0]]
    working_age += cols[2]
    return working_age
    
    

In [ ]:
df_sg['age_to_become_a_millionaire'] = df_sg[['Q4', 'Q6', 'millionaire_years']].apply(i_am_the_millionaire, axis=1)

The average age of the relevant fields when they finally hit that sweet million

In [ ]:
df_sg['age_to_become_a_millionaire'].groupby(df_sg['Q5']).mean()

We will filter out those who did not manage to hit a million dollars before the retirement age of 70

In [ ]:
df_sg_million = df_sg[df_sg['age_to_become_a_millionaire'] <= 70]

We have 74 possible Singaporean millionaires even if fsociety wipes their money!

In [ ]:
df_sg_million

**I then realised that Q1 actually tells the age range of each individual.... so once again..**

This time i will use Q1 (the age range) and the years it takes them to save a million

In [ ]:
def i_am_the_millionaire_rd_2(cols):
    x = cols[0].split('-')
    return (int(x[0]) + int(x[1]))/2 + cols[1]

In [ ]:
df_sg['age_to_become_a_millionaire_2'] = df_sg[['Q1', 'millionaire_years']].apply(i_am_the_millionaire_rd_2, axis=1)

In [ ]:
df_sg_million_2 = df_sg[df_sg['age_to_become_a_millionaire_2'] <= 70]

This time we have 71 possible millionaires!

In [ ]:
df_sg_million_2